# shubnikov de haas oscillations analysis




### python libraries and notes
* https://plotly.com/python/

* https://github.com/fbeilstein/machine_learning


We start with importing data from csv. Data was saved as **(field,T ; voltave, V)**

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots



def readSdH(csvName, current): #function to read and preprocess sdH oscillations 
    df = pd.read_csv(csvName, header=None)
    df.columns = ['field', 'voltage']

    df['resistance']= df['voltage']/current
    df['1/B'] = 1/df['field']

    return df

data4K =readSdH('data/sdH4K.csv',50* 10**(-9))
data10K =readSdH('data/sdH10K.csv',100* 10**(-9))

#defining columns for relative resistance oscillations and 1/B field
data4K['dR/R'] = (data4K['resistance']-data4K['resistance'][0])/data4K['resistance'][0]
data10K['dR/R'] = (data10K['resistance']-data10K['resistance'][0])/data10K['resistance'][0]


In [14]:
sdH4Kgraph = px.scatter(data4K, x = data4K['field'], y = data4K['dR/R'], width = 500, height = 300, template = 'simple_white')
sdH4Kgraph.update_layout(
    title=r'4K Mesa sdH oscillations',
    xaxis_title=r'Magnetic field, T',
    yaxis_title=r'$\frac{R_i-R(0)}{R(0)}$', font=dict(
        size=13
    ))
#markers style
sdH4Kgraph.update_traces(marker=dict(size=3,opacity=0.8, color = "#7AC5CD",
                              line=dict(width=1,
                                        color="#53868B")),
                  selector=dict(mode='markers'))

sdH4Kgraph.show()





In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = data4K['field'], y = data4K['dR/R'],
    name="4K"       # this sets its legend entry
))

fig.add_trace(go.Scatter(
    x = data10K['field'], y = data10K['dR/R'],
    name="10K"
))


fig.update_layout(height=400, width=600,template = 'simple_white', title_text="Stacked Subplots")
fig.update_layout(
    title=r'Mesa sdH oscillations',
    xaxis_title=r'Magnetic field, T',
    yaxis_title=r'$\frac{R_i-R(0)}{R(0)}$', font=dict(
        size=13
    )
)
fig.update_layout(legend_orientation="h")

fig.show()

## Getting $k_\text{F}$ from SdH oscillations

Here we define $f$ as the frequency of oscillations over $1/B$. 

The frequencies of the quantum oscillations of resistance $f$ are related to the cross section of the Fermi surfaces $A_{\text{k}}$ in the $k_{\parallel}$ plane by the Onsager relation:

$f = \frac{h}{4 \pi^2 e} A_{\text{k}} = \frac{h}{4 \pi e} k_{\parallel} ^2$


*Note*: we use almost free electron model, therefore, we consider Fermi surface to be a sphere. The section of the sphere is a circle. That means that $A_{\text{k}} = \pi k^2$.


First, we are getting the frequency by performing FFT on resistance oscillations over $1/B$


In [16]:
fig = px.scatter(data4K, x = data4K['1/B'], y = data4K['dR/R'], width = 500, height = 400, template = 'simple_white')
fig.update_layout(
    title=r'4K Mesa sdH oscillations',
    xaxis_title=r'Magnetic field, T',
    yaxis_title=r'$\frac{R_i-R(0)}{R(0)}$', font=dict(
        size=13
    ))
#markers style
fig.update_traces(marker=dict(size=1.5,opacity=1, color = "black",
                              line=dict(width=.5,
                                        color="#53868B")),
                  selector=dict(mode='markers'))

fig.show()